# Описание проекта
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.
Построим модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверим accuracy на тестовой выборке самостоятельно.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откроем и изучите файл

In [1]:
#Импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Откроем файл с данными
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
#Посмотрим первые 10 строк
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
#Изучим общую информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
#Заменим тип данных в столбцах 'messages' и 'calls' на int
df['messages'] = df['messages'].astype('int')
df['calls'] = df['calls'].astype('int')

In [6]:
#Проверим изменения
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null int64
minutes     3214 non-null float64
messages    3214 non-null int64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


In [7]:
#Посмотрим количество дубликатов
df.duplicated().sum()

0

### Вывод

С данными по большей части всё в порядке, нет пропусков и дубликатов, были только заменены столбцы "messages" и "calls" на тип "int".

## Разобьём данные на выборки

In [8]:
#Разобьём наши данные на тренировочную, валидационную и тестовую выборки
df_train, df_val = train_test_split(df, test_size=0.40, random_state=12345)
df_valid, df_test = train_test_split(df_val, test_size=0.50, random_state=12345)

In [9]:
#Посмотрим размер каждой выборки
print('Размер тренировочной выборки:', df_train.shape[0])
print('Размер валидационной выборки:', df_valid.shape[0])
print('Размер тестовой выборки:', df_test.shape[0])

Размер тренировочной выборки: 1928
Размер валидационной выборки: 643
Размер тестовой выборки: 643


In [10]:
#Составим признаки для каждой выборки
df_train_features = df_train.drop(['is_ultra'], axis=1)
df_train_target = df_train['is_ultra']
df_valid_features = df_valid.drop(['is_ultra'], axis=1)
df_valid_target = df_valid['is_ultra']
df_test_features = df_test.drop(['is_ultra'], axis=1)
df_test_target = df_test['is_ultra']

### Вывод

Данные разбили следующим образом:

- 60% данных - df_train;
- 20% данных - df_valid;
- 20% данных - df_test;

Обучение будет происходить на данных из df_train

Валидация будет происходить на данных из df_valid

Лучшая модель по валидации будет применена на данных df_test

## Исследуем модели

In [11]:
#Рассмотрим модель решающего дерева и выберем там наилучшую точность и глубину дерева
best_model_DTC = None
best_depth = 0
best_accuracy_DTC = 0
for depth in range(1, 101):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(df_train_features, df_train_target)
    predictions_valid = model.predict(df_valid_features)
    accuracy = accuracy_score(df_valid_target, predictions_valid)
    if accuracy > best_accuracy_DTC:
        best_model_DTC = model
        best_accuracy_DTC = accuracy
        best_depth = depth
print("Лучшая точность:", best_accuracy_DTC, "Глубина дерева:", best_depth)

Лучшая точность: 0.7853810264385692 Глубина дерева: 3


In [12]:
#Рассмотрим модель случайного леса и выберем там наилучшую точность и количество деревьев
best_model_RFC = None
best_est = 0
best_accuracy_RFC = 0

for est in range(1, 101):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(df_train_features, df_train_target)
    predictions_valid = model.predict(df_valid_features)
    accuracy = accuracy_score(df_valid_target, predictions_valid)
    if accuracy > best_accuracy_RFC:
        best_model_RFC = model
        best_accuracy_RFC = accuracy
        best_est = est
print("Лучшая точность:", best_accuracy_RFC, "Количество деревьев:", best_est)  

Лучшая точность: 0.7947122861586314 Количество деревьев: 23


In [13]:
##Рассмотрим модель логистической регрессии и выберем там наилучшую точность и итерацию
best_model_LR = None
best_accuracy_LR = 0
best_iteration = 0

for iteration in range(1, 101):
    model = LogisticRegression(random_state=12345)
    model.fit(df_train_features, df_train_target)
    predictions_valid = model.predict(df_valid_features)
    accuracy = accuracy_score(df_valid_target, predictions_valid)
    if accuracy > best_accuracy_LR:
        best_model_LR = model
        best_accuracy_LR = accuracy
        best_iteration = iteration
print("Лучшая точность:", best_accuracy_LR, "Итерация:", best_iteration)  

Лучшая точность: 0.7589424572317263 Итерация: 1


In [14]:
#Выпишем все наилучшие точности для каждой модели
print('DecisionTreeClassifier:', best_accuracy_DTC)
print('RandomForestClassifier:', best_accuracy_RFC)
print('LogisticRegression:', best_accuracy_LR)

DecisionTreeClassifier: 0.7853810264385692
RandomForestClassifier: 0.7947122861586314
LogisticRegression: 0.7589424572317263


### Вывод

Были исследованы модели решающего дерева, случайного леса и логистической регрессии.

Для модели решающего дерева:
- Лучшая точность: 0.7853810264385692 Глубина дерева: 3

Для модели случайного леса:
- Лучшая точность: 0.7947122861586314 Количество деревьев: 23

Для модели логистической регрессии:
- Лучшая точность: 0.7589424572317263 Итерация: 1

Наилучшая модель - модель случайного леса.

## Проверим модель на тестовой выборке

In [15]:
#Оценим точность каждой модели на тестовой выборке
test_predictions_DTC = best_model_DTC.predict(df_test_features)
accuracy_DTC = accuracy_score(df_test_target, test_predictions_DTC)
test_predictions_RFC = best_model_RFC.predict(df_test_features)
accuracy_RFC = accuracy_score(df_test_target, test_predictions_RFC)
test_predictions_LR = best_model_LR.predict(df_test_features)
accuracy_LR = accuracy_score(df_test_target, test_predictions_LR)

In [16]:
#Выведем точности каждой модели на экран
print('Точность модели решающего дерева на тестовой выборке:', accuracy_DTC)
print('Точность модели случайного леса на тестовой выборке:', accuracy_RFC)
print('Точность модели логистической регресиии на тестовой выборке:', accuracy_LR)

Точность модели решающего дерева на тестовой выборке: 0.7791601866251944
Точность модели случайного леса на тестовой выборке: 0.7807153965785381
Точность модели логистической регресиии на тестовой выборке: 0.7402799377916018


### Вывод

Были получены точности для каждой модели:
- Точность модели решающего дерева на тестовой выборке: 0.7791601866251944
- Точность модели случайного леса на тестовой выборке: 0.7807153965785381
- Точность модели логистической регресиии на тестовой выборке: 0.7402799377916018

В принципе, для всех моделей, кроме модели логистической регрессии accuracy > 0.75.

Если брать разницу между валидационной и тестовой выборкой для модели логистической регрессии, то там разница примерно в ~0.18, можно сделать вывод о слабой переобученности модели.

В итоге наилучшая модель на тестовой выборке - модель случайного леса.

## (бонус) Проверим модели на адекватность

Оценить модели на адекватность можно с помощью:

- accuracy - точность совпадения прогнозов с правильными ответами.
- precision - определяет, как много отрицательных ответов нашла модель, пока искала положительные. Чем больше отрицательных, тем ниже точность. Модель считается хорошей, если точность близка к единице.
- recall - Полнота выявляет, какую долю положительных среди всех ответов выделила модель. Полнота — это доля TP-ответов среди всех с истинной меткой «1». Хорошо, когда значение recall близко к единице: модель хорошо ищет положительные объекты. Если ближе к нулю — модель надо перепроверить и починить.
- F1-мера - объединение precision и recall.

Recall описывает, как хорошо модель разобралась в особенностях этого класса и распознала его. Precision выявляет, не переусердствует ли модель, присваивая положительные метки.

In [17]:
#Рассчитаем все метрики для модели решающего дерева
test_predictions_DTC = best_model_DTC.predict(df_test_features)
accuracy_DTC = accuracy_score(df_test_target, test_predictions_DTC)
precision_DTC = precision_score(df_test_target, test_predictions_DTC)
recall_DTC = recall_score(df_test_target, test_predictions_DTC)
f_score_DTC = f1_score(df_test_target, test_predictions_DTC)

In [18]:
#Рассчитаем все метрики для модели случайного леса
test_predictions_RFC = best_model_RFC.predict(df_test_features)
accuracy_RFC = accuracy_score(df_test_target, test_predictions_RFC)
precision_RFC = precision_score(df_test_target, test_predictions_RFC)
recall_RFC = recall_score(df_test_target, test_predictions_RFC)
f_score_RFC = f1_score(df_test_target, test_predictions_RFC)

In [19]:
#Рассчитаем все метрики для модели логистической регрессии
test_predictions_LR = best_model_LR.predict(df_test_features)
accuracy_LR = accuracy_score(df_test_target, test_predictions_LR)
precision_LR = precision_score(df_test_target, test_predictions_LR)
recall_LR = recall_score(df_test_target, test_predictions_LR)
f_score_LR = f1_score(df_test_target, test_predictions_LR)

In [20]:
#Выведем всё на экран
print('Для модели решающего дерева')
print('\n')
print('Accuracy_DTC:', accuracy_DTC, 'Precision_DTC:', precision_DTC, 'Recall_DTC:', recall_DTC, "F-мера:", f_score_DTC)
print('\n')
print('Для модели случайного леса')
print('\n')
print('Accuracy_RFC:', accuracy_RFC, 'Precision_RFC:', precision_RFC, 'Recall_RFC:', recall_RFC, "F-мера:", f_score_RFC)
print('\n')
print('Для модели логистической регрессии')
print('\n')
print('Accuracy_LR:', accuracy_LR, 'Precision_LR:', precision_LR, 'Recall_LR:', recall_LR, "F-мера:", f_score_LR)

Для модели решающего дерева


Accuracy_DTC: 0.7791601866251944 Precision_DTC: 0.744 Recall_DTC: 0.458128078817734 F-мера: 0.5670731707317074


Для модели случайного леса


Accuracy_RFC: 0.7807153965785381 Precision_RFC: 0.6781609195402298 Recall_RFC: 0.5812807881773399 F-мера: 0.6259946949602121


Для модели логистической регрессии


Accuracy_LR: 0.7402799377916018 Precision_LR: 0.8214285714285714 Recall_LR: 0.22660098522167488 F-мера: 0.3552123552123553


### Вывод

Адекватность модели оценивается с помощью accuracy, precision, recall, F-мера (объединение precision и recall). Наилучшая F-мера у модели случайного леса, как и accuracy с recall, при этом худший показатель precision по сравнению с 2 другими моделами.

## Общий вывод

С данными по большей части всё в порядке, нет пропусков и дубликатов, были только заменены столбцы "messages" и "calls" на тип "int".

Данные разбили следующим образом:

60% данных - df_train;
20% данных - df_valid;
20% данных - df_test;

Были исследованы модели решающего дерева, случайного леса и логистической регрессии.

Для модели решающего дерева:

Лучшая точность: 0.7853810264385692 Глубина дерева: 3

Для модели случайного леса:

Лучшая точность: 0.7947122861586314 Количество деревьев: 23

Для модели логистической регрессии:

Лучшая точность: 0.7589424572317263 Итерация: 1

Были получены точности для каждой модели:

- Точность модели решающего дерева на тестовой выборке: 0.7791601866251944
- Точность модели случайного леса на тестовой выборке: 0.7807153965785381
- Точность модели логистической регресиии на тестовой выборке: 0.7402799377916018

В принципе, для всех моделей, кроме модели логистической регрессии accuracy > 0.75.

Если брать разницу между валидационной и тестовой выборкой для модели логистической регрессии, то там разница примерно в ~0.18, можно сделать вывод о слабой переобученности модели.

Адекватность модели оценивается с помощью accuracy, precision, recall, F-мера (объединение precision и recall). Наилучшая F-мера у модели случайного леса, как и accuracy с recall, при этом худший показатель precision по сравнению с 2 другими моделами.

В целом можно сказать, что наилучшая модель из всех - это модель случайного леса, хотя она и самая медленная по скорости выполнения по сравнению с другими моделями.
